1. https://huggingface.co/blog/gemma
2. Repo model google/gemma-7b-it is gated. You must be authenticated to access it.
3. Create your 'new token' on your huggingface

pip install torch accelerate transformers langchain chroma sentence-transformers langchain_community pypdf

In [1]:
import os
from dotenv import load_dotenv
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig, pipeline
load_dotenv()

c:\Users\farza\anaconda3\envs\open_source_rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
GEMMA_TOKEN = os.getenv("GEMMA_TOKEN")
model = "google/gemma-7b-it"
device="cuda"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model, token=GEMMA_TOKEN, device=device)

quantization_config = GPTQConfig(
     bits=4,
     group_size=128,
     dataset="c4", # the original datasets used in GPTQ paper [‘wikitext2’,‘c4’,‘c4-new’,‘ptb’,‘ptb-new’]
     desc_act=False,
     tokenizer=tokenizer,
     batch_size=1,
)
quantized=False
if quantized:
    model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="google/gemma-7b-it",
                                                 token=GEMMA_TOKEN,
                                                 quantization_config=quantization_config,
                                                 device_map=device
                                                 )
else:
    model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="google/gemma-7b-it", 
                                                token=GEMMA_TOKEN,
                                                torch_dtype=torch.float16,
                                                device_map=device
                                                )

Loading checkpoint shards: 100%|██████████| 4/4 [00:11<00:00,  2.88s/it]


In [4]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

In [9]:
messages = [
    {"role": "user", "content": "tell me a joke"},
]
prompt = pipeline.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipeline(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
)
print(outputs[0]["generated_text"][len(prompt):])

Why did the tomato turn red?

Because it saw the salad dressing.
